In [ ]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
# import shutil
from pyspark.sql.types import *
import pyspark.sql.functions as Spark_f
from pyspark import SparkConf
from pyspark.sql.functions import format_number

In [3]:
# spark = SparkSession.builder.appName("PySparkAmazonReviewsTestSet").getOrCreate()
conf = SparkConf() \
    .setMaster("local[10]") \
    .set("spark.driver.memory", "2g") \
    .set("spark.executor.memory", "4g") \
    .set("spark.shuffle.spill", "true") \
    .set("spark.network.timeout", "60s") \
    .set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:G1ReservePercent=20") \
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")

spark = SparkSession.builder.config(conf=conf).appName("PySparkAmazonReviewsTestSet").getOrCreate()

24/11/25 19:43:40 WARN Utils: Your hostname, NBM-WXX9-7572c0be resolves to a loopback address: 127.0.1.1; using 192.168.88.79 instead (on interface wlp1s0)
24/11/25 19:43:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 19:43:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
def parse_custom_csv(line):
    import csv
    from io import StringIO

    try:
        return next(csv.reader(StringIO(line)))
    except:
        return ["PARSE_ERROR", line]
    
def to_date_(col, formats=("MM/dd/yyyy", "yyyy-MM-dd")):
    # Spark 2.2 or later syntax, for < 2.2 use unix_timestamp and cast
    return Spark_f.coalesce(*[Spark_f.to_date(col, f) for f in formats])

In [6]:
# init schema, str only
book_meta_struct = StructType([StructField("Title", StringType(), True),StructField("description", StringType(), True),StructField("authors", StringType(), True),StructField("image", StringType(), True), StructField("previewLink", StringType(), True), StructField("publisher", StringType(), True),StructField("publishedDate",StringType(),True),StructField("infoLink", StringType(), True), StructField("categories", StringType(), True), StructField("ratingsCount", StringType(), True)])
book_review_struct = StructType([StructField("Id",StringType(),True),StructField("Title",StringType(),True),StructField("Price",StringType(),True),StructField("User_id",StringType(),True),StructField("profileName",StringType(),True),StructField("review/helpfulness",StringType(),True),StructField("review/score",StringType(),True),StructField("review/time",StringType(),True),StructField("review/summary",StringType(),True),StructField("review/text",StringType(),True)])

In [ ]:
# pyspark криво парсит CSV файлы с кучей кавычек обычным способом, в некоторых случаях помогает опция unescapedQuoteHandling="STOP_AT_CLOSING_QUOTE", но не всегда справляется
raw_data = spark.sparkContext.textFile('//dataset/test/input/books_data.csv')
header = raw_data.first()
data_without_header = raw_data.filter(lambda line: line != header)
parsed_data = data_without_header.map(parse_custom_csv)

df = parsed_data.toDF(book_meta_struct)
# book_meta.withColumn("author_array",Spark_f.split(Spark_f.regexp_replace(Spark_f.col("authors"), r"[\[\]']", ""), ",")).show()
# book_meta = df.withColumns({"publishedDate":to_date_("publishedDate",formats=("yyyy-MM-dd","yyyy-MM","yyyy")),"ratingsCount":df.ratingsCount.cast(FloatType())})
book_meta = df.drop("ratingsCount").withColumns({"publishedDate":Spark_f.to_date(col("publishedDate")),"authors":Spark_f.split(Spark_f.regexp_replace(Spark_f.col("authors"), r"[\[\]']", ""), ","),"categories":Spark_f.split(Spark_f.regexp_replace(Spark_f.col("categories"), r"[\[\]']", ""), ",")})

book_meta.show(40)

24/11/25 19:43:50 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|Its Only Art If I...|                    |      [Julie Strain]|http://books.goog...|http://books.goog...|                    |   1996-01-01|http://books.goog...|[Comics & Graphic...|
|Dr. Seuss: Americ...|Philip Nel takes ...|        [Philip Nel]|http://books.goog...|http://books.goog...|           A&C Black|   2005-01-01|http://books.goog...|[Biography & Auto...|
|Wonderful Worship...|This resource inc...|      [David R. Ray]|http://books.goo

In [ ]:
raw_data = spark.sparkContext.textFile('/dataset/test/input/Books_rating.csv')
header = raw_data.first()
data_without_header = raw_data.filter(lambda line: line != header)
parsed_data = data_without_header.map(parse_custom_csv)

df = parsed_data.toDF(book_review_struct)
book_review = df.drop(df.Price).withColumns({"review/score":col("review/score").cast(FloatType()),"review/time":Spark_f.to_date(Spark_f.from_unixtime(col("review/time").cast("long")))})
# df = df.withColumns({"publishedDate":to_date_("publishedDate",formats=("yyyy-MM-dd","yyyy-MM","yyyy")),"ratingsCount":df.ratingsCount.cast(FloatType())})

book_review.show(40)

24/11/25 19:43:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+----------+--------------------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| AVCGYZL8FQQTD|Jim of Oz "jim-of...|               7/7|         4.0| 1999-10-23|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 2004-09-21|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 2004-03-09|Essential for eve...|If people become ...|
|0826414346|Dr. Seuss: Americ...|A2MVUWT453QH61|Roy E. Perry "am

24/11/25 19:43:56 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 3): Attempting to kill Python Worker


In [9]:
# book_meta_struct = StructType([StructField("Title", StringType(), True),StructField("description", StringType(), True),StructField("authors", ArrayType(StringType(),True), True),StructField("image", StringType(), True), StructField("previewLink", StringType(), True), StructField("publisher", StringType(), True), StructField("publishedDate",StringType(),True), StructField("infoLink", StringType(), True), StructField("categories", ArrayType(StringType(), True)), StructField("ratingsCount", FloatType(), True)])
# book_meta_struct = StructType([StructField("Title", StringType(), True),StructField("description", StringType(), True),StructField("authors", StringType(), True),StructField("image", StringType(), True), StructField("previewLink", StringType(), True), StructField("publisher", StringType(), True),StructField("publishedDate",DateType(),True),StructField("infoLink", StringType(), True), StructField("categories", StringType(), True), StructField("ratingsCount", FloatType(), True)])
# book_meta = spark.read.csv("/home/lshakhmetov/Загрузки/datasets/books_data.csv", schema=book_meta_struct,header=True, unescapedQuoteHandling="STOP_AT_CLOSING_QUOTE")
# book_meta = spark.read.csv("/home/lshakhmetov/Загрузки/datasets/books_data.csv", inferSchema=True,header=True)
# book_reviews = spark.read.csv("/home/lshakhmetov/Загрузки/datasets/Books_rating.csv",header=True)
# book_meta.show(30, True)
# book_reviews.show()

In [10]:
# book_meta = book_meta.withColumn("popularity",Spark_f.when(book_meta.Title == book_review.groupBy("Title").count()))
book_meta = book_meta.join(book_review.groupBy("Title").agg(Spark_f.count("*").alias("popularity")),on="Title", how="left").sort(Spark_f.desc("popularity")).limit(1000)
book_meta.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|         categories|popularity|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|          The Hobbit|Celebrating 75 ye...|  [J. R. R. Tolkien]|http://books.goog...|http://books.goog...|       Mariner Books|   2012-01-01|http://books.goog...| [Juvenile Fiction]|     22023|
| Pride and Prejudice|In early nineteen...|       [Jane Austen]|http://books.goog...|http://books.goog...| Courier Corporation|   1995-01-01|http://books.goog...|          [Fiction]|     20371|
|      Atlas Shrugged|        

In [11]:
book_review = book_review.join(book_meta,on='Title',how='leftsemi')
book_review.show()

+--------------------+----------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|        Id|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+----------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|Economics in one ...|B0007H4QBK|A2FYWUHFF21Q8F|          Mark Twian|               2/4|         5.0| 2004-02-09|    Short and Sweet.|For as important ...|
|Economics in one ...|B0007H4QBK| A5P9PV92PRYEE|gary@clearbridge.com|               2/4|         5.0| 1999-07-16|Explains why econ...|Hazlitt's view is...|
|Economics in one ...|B0007H4QBK|A1D4N3NVIN55PK|            fmj30cal|               0/1|         4.0| 2012-12-07|  Available for Free|You can get the f...|
|Economics in one ...|B0007H4QBK|A34AW9TMV7F69T|                

24/11/25 19:44:37 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 15 (TID 210): Attempting to kill Python Worker


In [12]:
book_meta.coalesce(1).write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/test/test_book_meta")
book_review.coalesce(1).write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/test/test_book_review")